In [ ]:
%pip install usf_fabric_monitoring

In [ ]:
from usf_fabric_monitoring.core.workspace_access_enforcer import WorkspaceAccessEnforcer
from usf_fabric_monitoring.core.logger import setup_logging
import logging
import os
from pathlib import Path

In [ ]:
import os
from dotenv import load_dotenv

# --- CREDENTIAL MANAGEMENT ---

# Option 1: Load from .env file in Lakehouse (Easiest migration)
# Upload your .env file to the 'Files' section of your Lakehouse
ENV_PATH = "/lakehouse/default/Files/.env"
if os.path.exists(ENV_PATH):
    print(f"Loading configuration from {ENV_PATH}")
    load_dotenv(ENV_PATH)
else:
    print(f"Warning: No .env file found at {ENV_PATH}")

# Option 2: Load from Azure Key Vault (Best Practice)
# Uncomment and configure this section to use Azure Key Vault
# try:
#     from notebookutils import mssparkutils
#     KEY_VAULT_NAME = "YourKeyVaultName"
#     os.environ["AZURE_CLIENT_ID"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Client-ID")
#     os.environ["AZURE_CLIENT_SECRET"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Client-Secret")
#     os.environ["AZURE_TENANT_ID"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Tenant-ID")
# except ImportError:
#     pass # Not running in Fabric or notebookutils not available
# except Exception as e:
#     print(f"Key Vault access failed: {e}")

# Verify credentials are present
required_vars = ["AZURE_CLIENT_ID", "AZURE_CLIENT_SECRET", "AZURE_TENANT_ID"]
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    print(f"❌ Missing required environment variables: {', '.join(missing)}")
else:
    print("✅ Credentials configured successfully")

In [ ]:
# Configuration
MODE = "assess" # "assess" or "enforce"
DRY_RUN = True # Set to False to actually apply changes when MODE is "enforce"
OUTPUT_DIR = Path("/lakehouse/default/Files/workspace_access_enforcement")
# OUTPUT_DIR = Path("exports/workspace_access_enforcement")

# Config files - assuming they are uploaded to the Lakehouse or available in the package
# For now, I'll define them inline or assume they are in the default location relative to the package if possible.
# But the package doesn't include config files by default unless specified in MANIFEST.in
# I'll define a simple default config here for demonstration.

REQUIREMENTS = {
    "default": [
        {"group_name": "Fabric Admins", "access_right": "Admin"}
    ]
}

SUPPRESSIONS = {
    "workspaces": []
}

In [ ]:
logger = setup_logging(name="workspace_enforcer", level=logging.INFO)

enforcer = WorkspaceAccessEnforcer(
    access_requirements=REQUIREMENTS,
    suppressions=SUPPRESSIONS,
    dry_run=DRY_RUN,
    logger=logger
)

summary = enforcer.enforce()

print(f"Enforcement Summary: {summary}")